# Step 1 - Preparatory processes

Log into accre (via a terminal window)

In [ ]:
ssh middlepg@login.accre.vanderbilt.edu

##### Step 1.1 - Check out specific versions of various toolboxes

_Goal_:  
Control the version of the toolboxes used; either update to most recent (present experiment), or update to correct commit (past experiment).

_Instructions_:  
Paste code into terminal. Make sure to clone versions of these toolboxes into your own home directory and refer to those paths.

_Output_:  
None

###IN ACCRE, Checkout the right commit of source codes (matlab and slurm scripts)

####If running a present experiment, update to the latest code:

In [ ]:
# Check out analysis code
cd /scratch/middlepg/perceptualchoice_stop_model/


git pull

# or

git fetch --all
# then
git reset --hard origin/master


####If re-running a past experiment, update to the correct commit (to be filled in by experimenter after updating):

In [ ]:
#Check out analysis code
cd /scratch/middlepg/perceptualchoice_stop_model/
git checkout <paste checkout version here> 
git checkout master;


###IN ACCRE, if needed checkout the right version of sam and related toolboxes.

Only do this if needed


In [1]:
# FIX THIS CODE TO OPTION WHETHER ON ACCRE OR LOCAL

#Check out a particular commit of sam toolbox, then the branch we were working on
cd /home/middlepg/m-files/sam/
git checkout xxxxx .
git checkout master;

#Check out my own MATLAB tools
cd /home/middlepg/m-files/matlab_code_bbz/
git checkout xxxxx .
git checkout master;

#Check out tools from the MATLAB File Exchange
cd /home/middlepg/m-files/matlab_file_exchange_tools/
git checkout xxxx .
git checkout master;

#Check out my cognitive modeling tools
cd /home/middlepg/m-files/cmtb/
git checkout xxxx .
git checkout master;


SyntaxError: invalid syntax (<ipython-input-1-943242c9d4e6>, line 1)

##### Step 1.2 - Add paths to MATLAB's search path

_Goal_:  
Ensure access to SAM toolbox and job code

_Instructions_:  
Paste code into MATLAB

_Output_:  
None

In [2]:
%%matlab
accreRoot = '/gpfs22';
accreHome = '/home/middlepg';
accreScratch = '/scratch/middlepg';
if isdir(fullfile(accreScratch))
    matRoot = fullfile(accreRoot,accreHome,'m-files');
    modelRoot = fullfile(accreScratch,'perceptualchoice_stop_model');
    environ = 'accre';
else
    matRoot = '/Volumes/HD-1/Users/paulmiddlebrooks/schalllab';
    modelRoot = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model';
    environ = 'local';
end

addpath(genpath(fullfile(matRoot,'sam')));
addpath(genpath(fullfile(matRoot,'matlab_code_bbz')));
addpath(genpath(fullfile(matRoot,'matlab_file_exchange_tools')));
addpath(genpath(fullfile(matRoot,'cmtb')));
addpath(genpath(fullfile(modelRoot,'src/code/2017-03-06')));

cd(fullfile(modelRoot,'src/code/2017-03-06/matlab'))


ERROR: Cell magic `%%matlab` not found.


##### Step 1.3 - Preprocess data

_Goal:_  
Convert behavioral data to SAM-compatible format

_Instruction_:  
Paste code into MATLAB

_Output_:  
`data_subj%.2d.mat` in `.../data/.../preproc01/subj%.2d/`

In [ ]:
%%matlab
rawDataDir          = fullfile(modelRoot,'data/raw/');
preprocDataDir      = fullfile(modelRoot,'data/2017-03-06/preproc01/');

subject             = 'broca';
sessions            = 'behavior2_ssd';
job_preproc_data(rawDataDir,subject,sessions,preprocDataDir);

subject             = 'xena';
sessions            = 'behavior1_ssd';
job_preproc_data(rawDataDir,subject,sessions,preprocDataDir);


##### Step 1.4 - Specify random number generator seed

_Goal_:  
Ensure reproducible findings

_Instruction_:  
Paste code into MATLAB

_Output_:  
`rngStruct.mat` in `.../data/.../preproc01/`

In [ ]:
%%matlab
preprocDataDir      = fullfile(modelRoot,'data/2017-03-06/preproc01/');
rngStruct           = rng('shuffle');
save(fullfile(preprocDataDir,'rngStruct.mat'),'rngStruct');



# Step 2 - Optimize the GO parameters, using a large step size

### Step 2.1 - Build the general SAM structure

_Goal_:  
Specify general settings, including details about the experiment, model, simulation, and optimization.

_Instruction_:  
Paste into MATLAB

_Output_:  
`SAM_goTrials.mat`

In [ ]:
%%matlab
subj                    = [1 2];
dt                      = 5;
trialVar                = true;
simScope                = 'go';
choiceMech              = {'race','ffi','li'};
stopMech                = 'none';
fileStr.rawDataDir      = fullfile(modelRoot,'data/raw/');
fileStr.preprocDataDir  = fullfile(modelRoot,'data/2017-03-06/preproc01/');
load(fullfile(fileStr.preprocDataDir,'rngStruct.mat'),'rngStruct');

nChoiceMech = numel(choiceMech);

for iChoiceMech = 1:nChoiceMech
    job_spec_sam_general(subj,dt,trialVar,simScope,choiceMech{iChoiceMech},stopMech,rngStruct,fileStr);
end


### Step 2.2 - Non-automatic optimization of GO parameters

_Goal_:  
To find good-fitting parameters by trial-and-error

_Instruction_:  
Paste code in notebook `Perceptual choice GO x0.ipynb` into MATLAB, in GUI mode. Tweak parameters until a satisfactory fit is obtained.

_Output_:  
`userSpecX_goTrials_model%.3d.mat`

### Step 2.3 - Automatic optimization of initial GO parameters

_Goal_:  
to find combinations of GO parameters with low Pearson's Chi-squared/BIC values that can be used as initial parameters for optimization

_Instruction_:  
Run the bash script `exe_job_optim_x0.txt` on ACCRE

_Output_:  
`finalLog_initParam_goTrials_model%.3d_started%s.mat`  
`SAM_initParam_goTrials_model%.3d_started%s.mat`  

####If running on ACCRE:

#### Update code in ACCRE by committing the most recent files/changes, and pulling in perceptualchoice_stop_model folder on ACCRE.

In [ ]:
%%system
scratch
cd perceptualchoice_stop_model/src/code/2017-03-06/sbatch
./exe_job_optim_x0.txt


####If running Locally:

In [ ]:
%%matlab
cd /Users/paulmiddlebrooks/perceptualchoice_stop_model/src/code/2017-03-06/matlab/
exe_job_optim_x0


##### Step 2.4 - Build model-specific SAM structures

_Goal_:  
to specify modeling settings that are specific to each model variant

_Instruction_:  
Paste into MATLAB  

_Output_:  
`SAM_goTrials_model%.3d.mat`

In [ ]:
subj                    = [1 2];
dt                      = 5;
trialVar                = true;
simScope                = 'go';
choiceMech              = {'race','li','ffi'};
stopMech                = 'none';
model                   = [2,43,79,352,478];
fileStr.root            = fullfile(modelRoot,'data/2017-03-06/preproc01/subj%.2d/dt%d/%s/%s/');
for iChoiceMech = 1:numel(choiceMech)
    job_spec_sam_model(subj,dt,trialVar,simScope,choiceMech{iChoiceMech},stopMech,model,fileStr);
end


##### Step 2.5 - Automatic optimization of GO parameters

_Goal_:  
Find best-fitting GO parameters using constrained Simplex algorithm

_Instruction_:  
Run the bash script `exe_job_optim_x.txt` on ACCRE

_Output_:  
`finalLog_goTrials_model%.3d_startVal%.3d_started%s.mat`  
`fValAndBestX_goTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  
`SAM_goTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  

####If running on ACCRE:

In [ ]:
%%system
scratch
cd perceptualchoice_stop_model/src/code/2017-03-06/sbatch
./exe_job_optim_x.txt


####If running Locally:

In [ ]:
%%matlab
cd /Users/paulmiddlebrooks/perceptualchoice_stop_model/src/code/2017-03-06/matlab/
exe_job_optim_x


##### Step 2.6 - Merge optimization files

_Goal_:  
Make one file containing minimized cost function value and best-fitting parameters

_Instruction_:  
Paste into MATLAB

_Output_:  
`allFValAndBestX_goTrials_model%.3d.txt`

In [ ]:
%%matlab
subj        = [1 2];
dt          = 5;
trialVar    = true;
optimScope  = 'go';
choiceMech  = {'race', 'li', 'ffi'};
stopMech    = 'none';
model                   = [2,43,79,352,478];

fileStr.root   = fullfile(modelRoot,'data/2017-03-06/preproc01/subj%.2d/dt%d/%s/%s/');
fileStr.output = 'allFValAndBestX_%sTrials_model%.3d.txt';

for iChoiceMech = 1:numel(choiceMech)
    job_merge_fvalandbestx(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech,model,fileStr);
end


##### Step 2.7 - Copy output to local computer

In [ ]:
%%system
scp -r middlepg@login.accre.vanderbilt.edu:/scratch/middlepg/perceptualchoice_stop_model/data/2017-03-06/preproc01/subj01/dt5/trialvar/* /Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2017-03-06/preproc01/subj01/dt5/trialvar/
    
scp -r middlepg@login.accre.vanderbilt.edu:/scratch/middlepg/perceptualchoice_stop_model/data/2017-03-06/preproc01/subj02/dt5/trialvar/* /Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2017-03-06/preproc01/subj02/dt5/trialvar/



# Stop Here. How are the GO fits??





Run code in Plot Model Fits notebook

# Step 3: Optimize STOP parameters

This involves fitting the stop-signal trials only. The GO parameters are fixed to the best-fitting values from Step 2.

Similar procedure as described above.

### Step 3 - Optimize the STOP parameters, using a large step size

##### Step 3.1 - Build the general SAM structure

_Goal_:  
Specify general settings, including details about the experiment, model, simulation, and optimization.

_Instruction_:  
Paste into MATLAB

_Output_:  
`SAM_stopTrials.mat`

In [ ]:
%%matlab
subj                    = [1 2];
dt                      = 5;
trialVar                = true;
simScope                = 'stop';
choiceMech              = {'race','li','ffi'};
stopMech                = 'li';
fileStr.rawDataDir      = fullfile(modelRoot,'data/raw/');
fileStr.preprocDataDir  = fullfile(modelRoot,'data/2017-03-06/preproc01/');
load(fullfile(fileStr.preprocDataDir,'rngStruct.mat'),'rngStruct');

nChoiceMech = numel(choiceMech);

for iChoiceMech = 1:nChoiceMech
    job_spec_sam_general(subj,dt,trialVar,simScope,choiceMech{iChoiceMech},stopMech,rngStruct,fileStr);
end


##### Step 3.2 - Non-automatic optimization of STOP parameters

_Goal_:  
To find good-fitting parameters by trial-and-error

_Instruction_:  
Paste code in notebook `Perceptual choice STOP x0.ipynb` into MATLAB, in GUI mode. Tweak parameters until a satisfactory fit is obtained.

_Output_:  
`userSpecX_stopTrials_model%.3d.mat`

##### Step 3.3 - Automatic optimization of initial STOP parameters

_Goal_:  
to find combinations of STOP parameters with low Pearson's Chi-squared/BIC values that can be used as initial parameters for optimization

_Instruction_:  
Run the bash script `exe_job_optim_x0.txt` on ACCRE

_Output_:  
`finalLog_initParam_stopTrials_model%.3d_started%s.mat`  
`SAM_initParam_stopTrials_model%.3d_started%s.mat`  

####If running on ACCRE:

%%system
scratch
cd perceptualchoice_stop_model/src/code/2017-03-06/sbatch
./exe_job_optim_x0_STOP_broca.txt

./exe_job_optim_x0_STOP_xena.txt


####If running Locally:

In [ ]:
%%matlab
cd /Users/paulmiddlebrooks/perceptualchoice_stop_model/src/code/2017-03-06/matlab/
exe_job_optim_x0_STOP


##### Step 3.4 - Build model-specific SAM structures

_Goal_:  
to specify modeling settings that are specific to each model variant

_Instruction_:  
Paste into MATLAB  

_Output_:  
`SAM_stopTrials_model%.3d.mat`

In [2]:
subj                    = [1 2];
model                   = [2,43,79,352,478];;
dt                      = 5;
trialVar                = true;
simScope                = 'stop';
choiceMech              = {'race','li','ffi'};
stopMech                = 'li';
fileStr.root   = fullfile(modelRoot,'data/2017-03-06/preproc01/subj%.2d/dt%d/%s/%s/');
for iChoiceMech = 1:numel(choiceMech)
    job_spec_sam_model(subj,dt,trialVar,simScope,choiceMech{iChoiceMech},stopMech,model,fileStr);
end


SyntaxError: invalid syntax (<ipython-input-2-bda021e3c66b>, line 9)

##### Step 3.5 - Automatic optimization of STOP parameters

_Goal_:  
Find best-fitting GO parameters using constrained Simplex algorithm

_Instruction_:  
Run the bash script `exe_job_optim_x.txt` on ACCRE

_Output_:  
`finalLog_stopTrials_model%.3d_startVal%.3d_started%s.mat`  
`fValAndBestX_stopTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  
`SAM_stopTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  

####If running on ACCRE:

%%system
scratch
cd perceptualchoice_stop_model/src/code/2017-03-06/sbatch
./exe_job_optim_x_STOP_broca.txt

./exe_job_optim_x_STOP_xena.txt


####If running Locally:

In [ ]:
%%matlab
cd /Users/paulmiddlebrooks/perceptualchoice_stop_model/src/code/2017-03-06/matlab/
exe_job_optim_x


##### Step 3.6 - Merge optimization files

_Goal_:  
Make one file containing minimized cost function value and best-fitting parameters

_Instruction_:  
Paste into MATLAB

_Output_:  
`allFValAndBestX_stopTrials_model%.3d.txt`

In [ ]:
%%matlab
subj        = [1 2];
dt          = 5;
trialVar    = true;
optimScope  = 'stop';
choiceMech  = {'race', 'li', 'ffi'};
stopMech    = 'li';
model       = [2,43,79,352,478];

fileStr.root   = fullfile(modelRoot,'data/2017-03-06/preproc01/subj%.2d/dt%d/%s/%s/');
fileStr.output = 'allFValAndBestX_%sTrials_model%.3d.txt';

for iChoiceMech = 1:numel(choiceMech)
    job_merge_fvalandbestx(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech,model,fileStr);
end


##### Step 3.7 - Copy output to local computer

In [ ]:
%%system
  
scp -r middlepg@login.accre.vanderbilt.edu:/scratch/middlepg/perceptualchoice_stop_model/data/2017-03-06/preproc01/subj01/dt5/trialvar/* /Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2017-03-06/preproc01/subj01/dt5/trialvar/

scp -r middlepg@login.accre.vanderbilt.edu:/scratch/middlepg/perceptualchoice_stop_model/data/2017-03-06/preproc01/subj02/dt5/trialvar/* /Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2017-03-06/preproc01/subj02/dt5/trialvar/


# Step 4 - Optimize the GO and STOP parameters

This involves fitting the go and stop trials. The GO and STOP parameters start at the best-fitting values from Step 2 and 3.

#### Step 4.1 - Build the general SAM structure

_Goal_:  
Specify general settings, including details about the experiment, model, simulation, and optimization.

_Instruction_:  
Paste into MATLAB

_Output_:  
`SAM_allTrials.mat`

In [2]:
%%matlab
subj                    = 1:2;
dt                      = 5;
trialVar                = true;
optimScope              = 'all';
choiceMech              = {'race','ffi','li'};
stopMech                = 'li';
fileStr.rawDataDir      = fullfile(modelRoot,'data/raw/');
fileStr.preprocDataDir  = fullfile(modelRoot,'data/2017-03-06/preproc01/');
load(fullfile(fileStr.preprocDataDir,'rngStruct.mat'),'rngStruct');

nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);

for iChoiceMech = 1:nChoiceMech
        job_spec_sam_general(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech,rngStruct,fileStr);
end


ERROR: Cell magic `%%matlab` not found.


##### Step 4.2 - Extract best-fitting GO and STOP parameters

In [ ]:
%%matlab
subj                    = 1:2;
dt                = 5;
trialVar                = true;
optimScope              = 'all';
stopMech                = {'li'};
model                   = [2,43,79,352,478];
fileStr.root            = fullfile(modelRoot,'data/2017-03-06/preproc01/subj%.2d/dt%d/%s/%s/');
fileStr.bestXSTOP       = fullfile(modelRoot,'data/2017-03-06/preproc01/subj%.2d/dt5/%s/%s/allFValAndBestX_stopTrials_model%.3d.txt');
x0Base                  = {};
doPlot                  = false;
doSave                  = true;
doStartParCluster       = false;


choiceMech  = {'race'};

nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);
nModel                  = numel(model);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        for iModel = 1:nModel
            job_spec_x0base(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model(iModel),fileStr,x0Base,doPlot,doSave,doStartParCluster);
        end
    end
end

choiceMech  = {'ffi'};

nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);
nModel                  = numel(model);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        for iModel = 1:nModel
            job_spec_x0base(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model(iModel),fileStr,x0Base,doPlot,doSave,doStartParCluster);
        end
    end
end

choiceMech  = {'li'}

nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);
nModel                  = numel(model);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        for iModel = 1:nModel
            job_spec_x0base(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model(iModel),fileStr,x0Base,doPlot,doSave,doStartParCluster);
        end
    end
end


##### Step 4.3 - Build model-specific SAM structure

_Goal_:  
to specify modeling settings that are specific to each model variant

_Instruction_:  
Paste into MATLAB  

_Output_:  
`SAM_allTrials_model%.3d.mat`

In [ ]:
%%matlab
subj                    = 1:2;
dt                      = 5;
trialVar                = true;
optimScope              = 'all';
choiceMech  = {'race', 'li', 'ffi'};
stopMech    = 'li';
model       = [2,43,79,352,478];
fileStr.root            = fullfile(modelRoot,'data/2017-03-06/preproc01/subj%.2d/dt%d/%s/%s/');

nChoiceMech             = numel(choiceMech);

for iChoiceMech = 1:nChoiceMech
        job_spec_sam_model(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech,model,fileStr);
end



##### Step 4.4 - Automatic optimization of GO and STOP parameters

_Goal_:  
Find best-fitting GO and STOP parameters using constrained Simplex algorithm

_Instruction_:  
Run the bash script `exe_job_optim_x.sh` on ACCRE

_Output_:  
`finalLog_allTrials_model%.3d_startVal%.3d_started%s.mat`  
`fValAndBestX_allpTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  
`SAM_allTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  

%%system
scratch
cd perceptualchoice_stop_model/src/code/2017-03-06/sbatch
./exe_job_optim_x_All_broca.txt
./exe_job_optim_x_All_xena.txt


##### Step 4.5 - Merge optimization files


_Goal_:  
Make one file containing minimized cost function value and best-fitting parameters

_Instruction_:  
Paste into MATLAB

_Output_:  
`allFValAndBestX_allTrials_model%.3d.txt`

In [ ]:
%%matlab
subj        = 1:2;
dt          = 5;
trialVar                = true;
optimScope              = 'all';
choiceMech  = {'race', 'li', 'ffi'};
stopMech    = {'li'};
model       = [2,43,79,352,478];
fileStr.root            = fullfile(modelRoot,'data/2017-03-06/preproc01/subj%.2d/dt%d/%s/%s/');


choiceMech  = {'race'};
nChoiceMech = numel(choiceMech);
nStopMech   = numel(stopMech);

fileStr.output = 'allFValAndBestX_%sTrials_model%.3d.txt';

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        job_merge_fvalandbestx(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

choiceMech  = {'ffi'};

for iChoiceMech = 1:numel(choiceMech)
    for iStopMech = 1:nStopMech
        job_merge_fvalandbestx(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

choiceMech  = {'li'};

for iChoiceMech = 1:numel(choiceMech)
    for iStopMech = 1:nStopMech
        job_merge_fvalandbestx(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

# Step 5 - Optimize GO and STOP parameters at 1-ms resolution


This involves fitting the go and stop trials. The GO and STOP parameters start at the best-fitting values from Step 4.

##### Step 5.1 - Build the general SAM structure

_Goal_:  
Specify general settings, including details about the experiment, model, simulation, and optimization.

_Instruction_:  
Paste into MATLAB

_Output_:  
`SAM_allTrials.mat`

In [ ]:
%%matlab
subj                    = 1:2;
dt                      = 1;
trialVar                = true;
optimScope              = 'all';
choiceMech              = {'race','ffi','li'};
stopMech                = {'li'};
fileStr.rawDataDir      = fullfile(modelRoot,'data/raw/');
fileStr.preprocDataDir  = fullfile(modelRoot,'data/2017-03-06/preproc01/');
load(fullfile(fileStr.preprocDataDir,'rngStruct.mat'),'rngStruct');

nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        job_spec_sam_general(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},rngStruct,fileStr);
    end
end


##### Step 5.2 - Extract best-fitting GO and STOP ("All") parameters at 5 ms resolution

In [ ]:
%%matlab
subj                    = 1:2;
dt                      = 1;
trialVar                = true;
optimScope              = 'all';
stopMech                = {'li'};
model                   = [2,43,79,352,478];
fileStr.root            = fullfile(modelRoot,'data/2017-03-06/preproc01/subj%.2d/dt%d/%s/%s/');
fileStr.bestXSTOP       = fullfile(modelRoot,'data/2017-03-06/preproc01/subj%.2d/dt5/%s/%s/allFValAndBestX_allTrials_model%.3d.txt');
x0Base                  = {};
doPlot                  = false;
doSave                  = true;
doStartParCluster       = false;

nStopMech               = numel(stopMech);
nModel                  = numel(model);


choiceMech              = {'race'};
nChoiceMech             = numel(choiceMech);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        for iModel = 1:nModel
            job_spec_x0base(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model(iModel),fileStr,x0Base,doPlot,doSave,doStartParCluster);
        end
    end
end


choiceMech  = {'ffi'};

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        for iModel = 1:nModel
            job_spec_x0base(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model(iModel),fileStr,x0Base,doPlot,doSave,doStartParCluster);
        end
    end
end


choiceMech  = {'li'}

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        for iModel = 1:nModel
            job_spec_x0base(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model(iModel),fileStr,x0Base,doPlot,doSave,doStartParCluster);
        end
    end
end


##### Step 5.3 - Build model-specific SAM structure

_Goal_:  
to specify modeling settings that are specific to each model variant

_Instruction_:  
Paste into MATLAB  

_Output_:  
`SAM_allTrials_model%.3d.mat`

In [ ]:
%%matlab
subj                    = 1:2;
dt                      = 1;
trialVar                = true;
optimScope              = 'all';
model                   = [2,43,79,352,478];
stopMech                = {'li'};
fileStr.root            = fullfile(modelRoot,'data/2017-03-06/preproc01/subj%.2d/dt%d/%s/%s/');


choiceMech              = {'race'};
nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        job_spec_sam_model(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

choiceMech              = {'ffi'};
for iChoiceMech = 1:numel(choiceMech)
    for iStopMech = 1:nStopMech
        job_spec_sam_model(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

choiceMech              = {'li'};
for iChoiceMech = 1:numel(choiceMech)
    for iStopMech = 1:nStopMech
        job_spec_sam_model(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end


##### Step 5.4 - Automatic optimization of GO and STOP parameters


_Goal_:  
Find best-fitting GO and STOP parameters using constrained Simplex algorithm

_Instruction_:  
Run the bash script `exe_job_optim_x.sh` on ACCRE

_Output_:  
`finalLog_allTrials_model%.3d_startVal%.3d_started%s.mat`  
`fValAndBestX_allpTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  
`SAM_allTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  

%%system
scratch
cd perceptualchoice_stop_model/src/code/2017-03-06/sbatch
./exe_job_optim_x_broca.txt

./exe_job_optim_x_xena.txt


##### Step 5.5 - Merge optimization files


_Goal_:  
Make one file containing minimized cost function value and best-fitting parameters

_Instruction_:  
Paste into MATLAB

_Output_:  
`allFValAndBestX_allTrials_model%.3d.txt`

In [ ]:
%%matlab
subj        = [1:2];
dt          = 1;
trialVar    = true;
optimScope  = 'all';
stopMech    = {'li'};
model       = [2,43,79,352,478];
fileStr.root  = fullfile(modelRoot,'data/2017-03-06/preproc01/subj%.2d/dt%d/%s/%s/');
fileStr.output = 'allFValAndBestX_%sTrials_model%.3d.txt';


choiceMech  = {'race'};
nChoiceMech = numel(choiceMech);
nStopMech   = numel(stopMech);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        job_merge_fvalandbestx(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

choiceMech  = {'ffi'};

for iChoiceMech = 1:numel(choiceMech)
    for iStopMech = 1:nStopMech
        job_merge_fvalandbestx(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

choiceMech  = {'li'};

for iChoiceMech = 1:numel(choiceMech)
    for iStopMech = 1:nStopMech
        job_merge_fvalandbestx(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end


##### Step 5.2 - Extract best-fitting GO and STOP parameters at 1 ms resolution

In [ ]:
%%system
mkdir /Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2017-03-06/preproc01/subj01/dt1/
scp -r middlepg@login.accre.vanderbilt.edu:/scratch/middlepg/perceptualchoice_stop_model/data/2017-03-06/preproc01/subj01/dt1/* /Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2017-03-06/preproc01/subj01/dt1/

mkdir /Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2017-03-06/preproc01/subj02/dt1/
scp -r middlepg@login.accre.vanderbilt.edu:/scratch/middlepg/perceptualchoice_stop_model/data/2017-03-06/preproc01/subj02/dt1/* /Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2017-03-06/preproc01/subj02/dt1/
        
